In [2]:
import pandas as pd
import glob
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["SCIPY_ARRAY_API"] = "1"

In [3]:
# путь к папке
folder = "data/wells"

# ищем все csv-файлы
all_csv = glob.glob(os.path.join(folder, "*.csv"))

# читаем и объединяем
df_0 = pd.concat((pd.read_csv(f) for f in all_csv), ignore_index=True)
print(f"Считано файлов: {len(all_csv)}")
# переименуем для удобства
df_0 = df_0.rename(columns={"BBLS_OIL_COND":"oil", "MCF_GAS": 'gas', "BBLS_WTR":"water", "API_WellNo":"well_name", "RptDate":"date", "DAYS_PROD":"days_prod"})
# преобразуем колонку RptDate к datetime
df_0["date"] = pd.to_datetime(df_0["date"])

Считано файлов: 4485


In [4]:
df = df_0.drop(columns=["Lease_Unit", "Formation"])
print(df.shape)

(1212779, 6)


In [5]:
# важно, чтобы внутри каждой скважины ряды шли строго по времени
df = df.sort_values(by=["well_name", "date"]).reset_index(drop=True)

In [6]:
# Есть записи с отрицательными дебитами. Исключим их, так как работа будет вестись только с добывающими скважинами
df = df[(df['oil'] >= 0) & (df['gas'] >= 0) & (df['water'] >= 0)]

In [7]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Предобработка профилей (Шаг 1)
------------------------------------------------------------------
Этот блок определяет конфиг и функции предобработки для датафрейма df
со столбцами:
['well_name', 'date', 'oil', 'water', 'gas', 'days_prod']

Основные шаги:
- Приведение к среднесуточным дебитам (учёт days_prod, иначе — дни в месяце)
- Единая месячная сетка и агрегация
- Фильтры качества и робастная обработка выбросов
- Сглаживание (Savitzky–Golay)
- Производные каналы (wc, gor), нормализация по пику
- Выравнивание времени (t=0 — старт производства)
- Усечение/паддинг до горизонта T
- Подготовка длинного и «tensor» представления

Код написан так, чтобы безопасно выполняться даже без df в окружении.
"""

from tools.preprocessing import *

if 'df' in globals():
    print("Обнаружен df в окружении. Запускаю предобработку с дефолтным конфигом...")
    cfg = PreprocConfig()
    out = preprocess_profiles(df, cfg)
    # Покажем заголовок long-таблицы как предварительный просмотр
    head_preview = out["panel_long"].head(12).copy()
    try:
        # Если доступна вспомогательная функция отображения таблиц — используем её
        import caas_jupyter_tools
        caas_jupyter_tools.display_dataframe_to_user("Предпросмотр panel_long", head_preview)
    except Exception:
        # Иначе просто печатаем
        print(head_preview)
else:
    print("Готово. Функции предобработки загружены. Для запуска вызовите:")
    print("cfg = PreprocConfig(T=36, min_profile_months=12)")
    print("out = preprocess_profiles(df, cfg)")
    print("panel_long, X = out['panel_long'], out['X']")


Обнаружен df в окружении. Запускаю предобработку с дефолтным конфигом...
Preprocess complete.
  В итог попало скважин: 2142
  Отброшено (по причинам):


,reason,count
0,no_start_detected,1930
1,too_short(<12),51


             well_name       date  oil_sum  water_sum  gas_sum  days_sum  \
0   25-003-05007-00-00 1986-01-01    252.0     2552.0      0.0      31.0   
1   25-003-05007-00-00 1986-02-01    239.0     2295.0      0.0      28.0   
2   25-003-05007-00-00 1986-03-01    253.0     2697.0      0.0      31.0   
3   25-003-05007-00-00 1986-04-01    249.0     1883.0      0.0      30.0   
4   25-003-05007-00-00 1986-05-01    278.0     2631.0      0.0      31.0   
5   25-003-05007-00-00 1986-06-01    244.0     2086.0      0.0      30.0   
6   25-003-05007-00-00 1986-07-01    271.0     2322.0      0.0      31.0   
7   25-003-05007-00-00 1986-09-01    195.0     1625.0      0.0       0.0   
8   25-003-05007-00-00 1986-10-01    224.0     1867.0      0.0       0.0   
9   25-003-05007-00-00 1986-11-01    209.0     1742.0      0.0       0.0   
10  25-003-05007-00-00 1986-12-01    224.0     1867.0      0.0       0.0   
11  25-003-05007-00-00 1987-01-01    221.0     2066.0      0.0       0.0   

    valid_d

In [ ]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Шаг 2 (признаки) и Шаг 3 (Manifold)
-----------------------------------------------------------------------
Зависимости (pip): numpy, pandas, scikit-learn, umap-learn, fastdtw, tqdm (опц.)

Ожидается, что из Шага 1 у вас уже есть объект `out` с ключами:
  - out["panel_long"]: long-таблица по скважинам (после предобработки)
  - out["X"]: np.ndarray формы [n_wells, T, C] по каналам cfg.tensor_channels
  - out["wells_used"]: список имён скважин в том же порядке, что и ось 0 в X
  - out["tensor_channels"]: список имён каналов в X (например: ["r_oil_norm","wc","gor","dr_oil_norm"]) 

Этот файл добавляет:
  1) Шаг 2: компактные дескрипторы профилей (side features)
  2) Шаг 3: построение низкоразмерной "карты поведения" (UMAP) двумя способами:
       a) быстрый базовый (евклид на выровненных рядах)
       b) уточнённый с FastDTW, без полной O(N^2) — через доуточнение k ближайших пар

Примечание: библиотеку dtaidistance *не* используем. Для DTW берём fastdtw.
"""
from __future__ import annotations
from tools.manifold import *
from tools.feature import *

try:
    from fastdtw import fastdtw
except Exception:
    fastdtw = None
    warnings.warn("fastdtw не найден. DTW-вариант будет недоступен, останется базовый евклид.")

# tqdm (опционально)
try:
    from tqdm import tqdm
    _TQDM = True
except Exception:
    _TQDM = False


if 'out' in globals():
    panel_long = out["panel_long"]
    X = out["X"]
    wells = out["wells_used"]
    tensor_channels = out["tensor_channels"]
    T = out["config"]["T"] if "config" in out else X.shape[1]

    # --- Шаг 2: компактные фичи
    feats = compute_side_features(panel_long, T=T)
    feats_scaled, scaler = scale_features(feats)
    print("Side features shape:", feats_scaled.shape)

    # --- Шаг 3a: быстрый UMAP по евклиду на рядах
    Z_euclid, umap_e = embed_umap_euclid(
        X, tensor_channels=tensor_channels, channels=["r_oil_norm", "wc"],
        n_neighbors=30, min_dist=0.05, n_components=2, random_state=42
    )

    # --- Шаг 3b: UMAP с FastDTW-уточнением
    cfg_m = ManifoldConfig(channels=("r_oil_norm", "wc"), fastdtw_radius=6, k_refine=40,
                           weights=(0.7, 0.3), n_neighbors=30, min_dist=0.05,
                           n_components=2, random_state=42)
    Z_dtw, sub_idx, D, info = embed_umap_fastdtw(
        X, tensor_channels=tensor_channels, channels=cfg_m.channels,
        cfg=cfg_m, sample_size=1000  # можно None для полного набора
    )
    print("DTW-UMAP:", Z_dtw.shape, "subset size:", len(sub_idx))

    # Пример: собрать DataFrame с координатами
    df_map = pd.DataFrame({
        "well_name": np.array(wells)[sub_idx],
        "x": Z_dtw[:,0],
        "y": Z_dtw[:,1],
    })
    display(df_map.head())
else:
    print("Для примера использования требуется объект 'out' из Шага 1.")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Side features shape: (2142, 16)


Recompute DTW for 29246 pairs (radius=6):   5%|▌         | 1512/29246 [00:17<05:55, 78.09it/s]

In [15]:
from tools.clustering import *

if 'out' in globals() and 'Z_dtw' in globals() and 'sub_idx' in globals():
    panel_long = out["panel_long"]
    wells_all = out["wells_used"]
    T = out["config"].get("T", 36)
    wells_sub = np.array(wells_all)[sub_idx].tolist()

    # 1) HDBSCAN
    cfg = ClusterConfig(min_cluster_size=50, min_samples=12)
    res = cluster_hdbscan(Z_dtw, wells_sub, cfg)
    df_map = res["df_map"]
    print("Silhouette:", res["silhouette"], " DBCV:", res["dbcv"])
    print(summarize_clusters(df_map))

    # 2) Аномалии
    df_map = assign_anomaly_scores(df_map, Z_dtw, res["labels"], lof_k=30)
    # пример отбора аномалий
    anomalies = df_map.sort_values("anomaly_score", ascending=False).head(30)

    # 3) Прототипы (медианные или soft-DTW / DBA при наличии tslearn)
    protos = build_cluster_prototypes(panel_long, df_map, channels=("r_oil_s","wc","gor","r_oil_norm"), T=T, method="auto")

    # Теперь df_map можно соединить с геолого-технологическими признаками и строить объяснения.
else:
    print("Для примера нужен 'out' (Шаг 1) и 'Z_dtw, sub_idx' (Шаг 3).")


Silhouette: nan  DBCV: 0.0
   cluster  size  share  prob_median
0       -1   100    1.0          0.0


In [16]:
from tools.make_reports import *

panel_long = out["panel_long"]
T = out["config"]["T"]
df_map = res["df_map"]  # из HDBSCAN/GMM
out_dir = "reports/pbm_report_exports"

map_png   = save_pbm_map(Z_dtw, df_map, out_dir)              # карта PBM
sizes_png = save_cluster_distribution_plot(df_map, out_dir)   # размеры кластеров
proto_pngs = save_cluster_prototype_plots(
    panel_long, df_map, protos, channels=("r_oil_s","wc","gor","r_oil_norm"), T=T, out_dir=out_dir
)

summary = summarize_clusters(df_map)
csvs = export_csv_summaries(df_map, summary, out_dir, top_anoms=50)

report = build_html_report(out_dir, map_png, sizes_png, proto_pngs, df_map, summary, title="PBM Report")
print("Отчёт:", report)

Отчёт: reports/pbm_report_exports/PBM_report.html



# Шаг 6. Прогноз профиля по префиксу (20 → 100)

**Цель:** прогнозировать месяцы 21–100 для каждой скважины, используя только первые 20 месяцев и без утечки информации.

Подходы в этом шаге:
1. **KNN-достройка по префиксу** с амплитудным выравниванием соседей (по МНК на префиксе).
2. **Мультивыходная ElasticNet-регрессия** на компактных признаках префикса.

Метрики: RMSE и sMAPE на окне 21–100.  
Отчёты и артефакты сохраняются в `./forecast_exports`.


In [17]:
# === Шаг 6: Прогноз профиля по префиксу (20 → 100) ===
from tools.forecast import *
import os, numpy as np, pandas as pd, json
from datetime import datetime
import matplotlib.pyplot as plt

assert 'out' in globals(), "Требуется объект 'out' из Шага 1 (preprocess_profiles)."
panel_long = out["panel_long"].copy()
wells_used = out["wells_used"]
T = int(out["config"]["T"])
T_pref = 20  # можно вынести в конфиг

# 6.1. Построить префикс-нормированный канал без утечки
panel_long = build_prefix_scaled_channel(panel_long, wells_used, T=T, T_pref=T_pref,
                                         q=0.90, rate_col="r_oil_s", out_col="r_oil_pref_norm")

# 6.2. Матрицы X_pref, Y_suffix, Y_full
X_pref, Y_suffix_true, Y_full = make_matrices(panel_long, wells_used, T=T, T_pref=T_pref,
                                              channel="r_oil_pref_norm", target_col="r_oil_s")

# 6.3. KNN-достройка
Y_pred_knn, knn_info = knn_forecast(X_pref, Y_full, T_pref=T_pref, K=15)

# 6.4. Мультивыходная регрессия
Y_pred_lr, lr_info = multioutput_forecast(panel_long, wells_used, T=T, T_pref=T_pref, Y_full=Y_full, random_state=43)

# 6.5. Оценка качества
m_knn = evaluate_forecasts(Y_suffix_true, Y_pred_knn)
m_lr  = evaluate_forecasts(Y_suffix_true, Y_pred_lr)
print("KNN   → RMSE={rmse:.4f}, sMAPE={smape:.4f}, N={n_eval}".format(**m_knn))
print("ENet  → RMSE={rmse:.4f}, sMAPE={smape:.4f}, N={n_eval}".format(**m_lr))

# 6.6. Сохранение прогнозов и отчёта
out_dir = "reports/forecast_exports"
os.makedirs(out_dir, exist_ok=True)

np.save(os.path.join(out_dir, "Y_suffix_true.npy"), Y_suffix_true)
np.save(os.path.join(out_dir, "Y_pred_knn.npy"), Y_pred_knn)
np.save(os.path.join(out_dir, "Y_pred_enet.npy"), Y_pred_lr)

# Таблица метрик
metrics_df = pd.DataFrame([
    {"model": "knn", "rmse": m_knn["rmse"], "smape": m_knn["smape"], "n_eval": m_knn["n_eval"]},
    {"model": "elasticnet", "rmse": m_lr["rmse"], "smape": m_lr["smape"], "n_eval": m_lr["n_eval"]},
])
metrics_csv = os.path.join(out_dir, "metrics.csv")
metrics_df.to_csv(metrics_csv, index=False)

# Пер-скважинные прогнозы (с именами)
def save_predictions_csv(Y_pred: np.ndarray, wells: list, name:str):
    cols = [f"m{t}" for t in range(T_pref+1, T+1)]
    df = pd.DataFrame(Y_pred, columns=cols)
    df.insert(0, "well_name", wells)
    df.to_csv(os.path.join(out_dir, f"pred_{name}.csv"), index=False)

save_predictions_csv(Y_pred_knn, wells_used, "knn")
save_predictions_csv(Y_pred_lr,  wells_used, "elasticnet")

# 6.7. Примеры графиков "факт vs прогноз"
def plot_example(idx, title, ytrue, ypred):
    plt.figure()
    plt.plot(range(T_pref, T), ytrue[idx], label="true")
    plt.plot(range(T_pref, T), ypred[idx], label="pred")
    plt.title(title)
    plt.xlabel("month index")
    plt.ylabel("oil rate (r_oil_s)")
    plt.legend()
    fig_path = os.path.join(out_dir, f"{title.replace(' ','_').lower()}_{idx}.png")
    plt.savefig(fig_path, dpi=140, bbox_inches="tight")
    plt.close()
    return fig_path

# случайные 3 примера из обучаемых скважин
rng = np.random.default_rng(43)
I = np.where(np.isfinite(Y_pred_knn).all(axis=1))[0]
show = rng.choice(I, size=min(3, len(I)), replace=False) if len(I) else []
example_imgs = []
for i in show:
    example_imgs.append(plot_example(i, "knn_example", Y_suffix_true, Y_pred_knn))
    example_imgs.append(plot_example(i, "enet_example", Y_suffix_true, Y_pred_lr))

# 6.8. Простой HTML отчёт
html = f"""
<html><head><meta charset='utf-8'><title>Forecast Report</title></head><body>
<h2>Forecast evaluation (prefix {T_pref} → total {T})</h2>
<p>Generated: {datetime.utcnow().isoformat()}Z</p>
<table border='1' cellspacing='0' cellpadding='6'>
<tr><th>Model</th><th>RMSE</th><th>sMAPE</th><th>N eval wells</th></tr>
<tr><td>KNN</td><td>{m_knn['rmse']:.4f}</td><td>{m_knn['smape']:.4f}</td><td>{m_knn['n_eval']}</td></tr>
<tr><td>ElasticNet</td><td>{m_lr['rmse']:.4f}</td><td>{m_lr['smape']:.4f}</td><td>{m_lr['n_eval']}</td></tr>
</table>
<h3>Files</h3>
<ul>
  <li>metrics.csv</li>
  <li>pred_knn.csv</li>
  <li>pred_elasticnet.csv</li>
</ul>
<h3>Examples</h3>
{''.join(f"<img src='{os.path.basename(p)}' style='max-width:640px;display:block;margin-bottom:10px;'/>" for p in example_imgs)}
</body></html>
"""
report_path = os.path.join(out_dir, "forecast_report.html")
with open(report_path, "w", encoding="utf-8") as f:
    f.write(html)

print("Saved:", metrics_csv, "and", report_path)


KNN   → RMSE=20.3636, sMAPE=0.4884, N=1964
ENet  → RMSE=36.7395, sMAPE=1.1923, N=1964
Saved: reports/forecast_exports/metrics.csv and reports/forecast_exports/forecast_report.html
